### ДИКЛЕЙМЕР
Этот код проба только для объединенного файла таблиц xlsx, который можно потом в принципе переделать под любой другой 

### Сортировка по алфавиту

За основу взят столбец Full Name


In [16]:
import pandas as pd 

csvData = pd.read_csv("./xlsx_data.csv") 
                                         
# sort data frame 
csvData.sort_values(["Full Name"],  
                    axis=0, 
                    #ascending=[False],  
                    inplace=True) 
  
csvData.head(10)


/tmp/ipykernel_21001/3690000242.py:3: DtypeWarning: Columns (0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,27,28,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  csvData = pd.read_csv("./results.csv")


,PassengerFirstName\tPassengerSecondName\tPassengerLastName\tPassengerSex\tPassengerBirthDate\tPassengerDocument\tBookingCode\tTicketNumber\tBaggage\tFlightDate\tFlightTime\tFlightNumber\tCodeShare\tDestination,AgentInfo,PaxAdditionalInfo,Meal,Seat,ArrivalDate,ArrivalTime,TrvCls,First Name,Last Name,...,Birth Date,FlightTime,BookingCode,TicketNumber,Baggage,Second Name,User uid,BonusProgram,Full Name,SEQUENCE
2504916,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12:35,XTMOFM,3.193258e+15,NaN,NaN,NaN,NaN,ABRAMOV AL'BERT,19.0
2491369,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,07:17,JSSUJO,3.880300e+15,NaN,NaN,NaN,NaN,ABRAMOV ALEKSEI,21.0
2556065,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,08:15,DZZMYI,6.695653e+15,NaN,NaN,NaN,NaN,ABRAMOV ALEXEY R,26.0
2548971,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17:50,SZHUJN,4.324225e+15,NaN,NaN,NaN,NaN,ABRAMOV ANATOLII,22.0
2659146,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,23:59,KEAYZQ,8.465826e+15,NaN,NaN,NaN,NaN,ABRAMOV ANATOLII,98.0
2303407,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12:40,NDRSWI,5.694357e+15,NaN,NaN,NaN,NaN,ABRAMOV ANATOLIY,75.0
2263781,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,08:00,DZMSZC,3.461786e+15,NaN,NaN,NaN,NaN,ABRAMOV ANATOLIY M,77.0
2645687,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15:50,QWHAIC,7.262463e+15,NaN,NaN,NaN,NaN,ABRAMOV ANATOLIY M,98.0
2445702,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10:00,OCOMHI,8.802057e+15,NaN,NaN,NaN,NaN,ABRAMOV ANTON,7.0
2517485,NaN,NaN,Boarding is ended 20 minutes before departure ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,07:00,ZLLPVS,8.082011e+15,NaN,NaN,NaN,NaN,ABRAMOV ANTON,49.0


### Сортировка по совпадению документов

Было принято решение обрабатывать файлы по отдельности, так как не во всех файлах существуют документы хоть в каком-нибудь виде.

1. BoardingData.csv

In [7]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('BoardingData.csv', sep=';')

# Удаляем пробелы в названиях столбцов
df.columns = df.columns.str.strip()

filtered_df = (df.groupby('PassengerDocument')
               .filter(lambda x: x['PassengerFirstName'].nunique() > 1 or x['PassengerLastName'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates()

if final_df.empty:
    print("Совпадений не найдено")
else:
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersCSV.csv', index=False) 
    

       PassengerFirstName PassengerSecondName PassengerLastName PassengerSex  \
83246              GORDEI                  M.            GLEBOV         Male   
92874              GORDEI                  M.            GLEBOV         Male   
134532             GORDEI          MAKAREVICH            GLEBOV         Male   
150151              RAMIL         ALBERTOVICH             BUROV         Male   

       PassengerBirthDate PassengerDocument    BookingCode      TicketNumber  \
83246          12/31/1982       8248 013778  Not presented  5858732412927507   
92874          12/31/1982       8248 013778         EJQBAD  2082229451418171   
134532         12/31/1982       8248 013778         BQCUKX     Not presented   
150151         08/24/1995       8248 013778  Not presented  7230123462656045   

           Baggage  FlightDate FlightTime FlightNumber CodeShare Destination  
83246   Registered  2017-03-09      07:15       SU1276       Own       Kazan  
92874      Transit  2017-01-16      01:4

2. JSON. Сначала перевод в CSV для удобства. Файл - FrequentFlyerForum-Profiles.json

In [5]:
import pandas as pd
import json
with open('./data/Airlines/FrequentFlyerForum-Profiles.json', 'r') as f:
    data = json.load(f)

# Преобразование данных JSON в pandas DataFrame
profiles = data["Forum Profiles"]

parsed_data = []
for profile in profiles:
    first_name = profile["Real Name"]["First Name"]
    last_name = profile["Real Name"]["Last Name"]
    sex = profile["Sex"]
    document = profile["Travel Documents"][0]["Passports"]
    
    for flight in profile["Registered Flights"]:
        if profile.get("Loyality Programm"):
            loyality_program = profile["Loyality Programm"][0]
            loy_prgrm_status = loyality_program.get("Status", None)
            loy_prgrm_code = loyality_program.get("programm", None)
            loy_prgrm_number = loyality_program.get("Number", None)
        else:
            loy_prgrm_status = loy_prgrm_code = loy_prgrm_number = None
        
        parsed_data.append({
            "First Name": first_name,
            "Last Name": last_name,
            "Sex": sex,
            "Document": document,
            "Flight Date": flight["Date"],
            "Flight Number": flight["Flight"],
            "Share Code": flight["Codeshare"],
            "ArrivalCity": flight["Arrival"]["City"],
            "ArrivalAirport": flight["Arrival"]["Airport"], 
            "DepartureCity": flight["Departure"]["City"],   
            "DepartureAirport": flight["Departure"]["Airport"],
            "NickName": profile.get("NickName", None), 
            "LoyPrgrmStatus": loy_prgrm_status,
            "LoyPrgrmCode": loy_prgrm_code,
            "LoyPrgrmNumber": loy_prgrm_number
        })

# Преобразование списка в pandas DataFrame
df_json = pd.DataFrame(parsed_data)
df_json.to_csv('FrequentFlyerForum-Profiles-json.csv', index=False)

In [10]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('FrequentFlyerForum-Profiles-json.csv', sep=',')

# Удаляем пробелы в названиях столбцов
df.columns = df.columns.str.strip()

filtered_df = (df.groupby('Document')
               .filter(lambda x: x['First Name'].nunique() > 1 or x['Last Name'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates()

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersJSON.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

Совпадений не найдено


3. Sirena-export-fixed.tab. Схема та же, что и с JSON.

In [14]:
colspecs = [
    (0, 60),   # PaxName
    (60, 72),  # PaxBirthDate
    (72, 84),  # DepartDate
    (84, 96),  # DepartTime
    (96, 108),  # ArrivalDate
    (108, 120),  # ArrivalTime
    (120, 126),  # Flight
    (126, 132),  # CodeSh
    (132, 138),  # From
    (138, 144),  # Dest
    (144, 150),  # Code
    (150, 168),  # e-Ticket
    (168, 180), # TravelDoc
    (180, 186),# Seat
    (186, 192),# Meal
    (192, 198),# TrvCls
    (198, 216),# Fare  Baggage
    (216, 240),# PaxAdditionalInfo
    (240, 246),# Unnamed: 18
    (246, 276), # Unnamed: 19
    (276, 336) # AgentInfo
]

df_tab = pd.read_fwf('./data/Airlines/Sirena-export-fixed.tab', colspecs=colspecs, dtype={'Unnamed: 19': 'str', 'Unnamed: 18': 'str'})

# Удаляем 'FF#' в столбце Unnamed: 18, оставляем только программный код (например, 'DT')
df_tab['Unnamed: 18'] = df_tab['Unnamed: 18'].str.replace('FF#', '', regex=False)

# Шаг 4: Разделение PaxName на три части (Last Name, First Name, Second Name)
#df_tab[['Last Name', 'First Name', 'Second Name']] = df_tab['PaxName'].str.split(expand=True, n=2)

# Шаг 5: Переименование и объединение столбцов из Sirena с основным датасетом
df_tab = df_tab.rename(columns={
    'PaxBirthDate': 'Birth Date',
    'Flight': 'Flight Number',
    'CodeSh': 'Share Code',
    'From': 'DepartureAirport',
    'Dest': 'ArrivalAirport',
    'Code': 'BookingCode',
    'e-Ticket': 'TicketNumber',
    'TravelDoc': 'Document',
    'Fare  Baggage': 'Baggage',
    'DepartDate': 'Flight Date',
    'DepartTime': 'FlightTime',
    'Unnamed: 19': 'LoyPrgrmNumber',
    'Unnamed: 18': 'LoyPrgrmCode'
})

df_tab.to_csv('Sirena-export-fixed-tab.csv', index=False)

In [15]:
df_tab.head(10)

,PaxName,Birth Date,Flight Date,FlightTime,ArrivalDate,ArrivalTime,Flight Number,Share Code,DepartureAirport,ArrivalAirport,...,TicketNumber,Document,Seat,Meal,TrvCls,Baggage,PaxAdditionalInfo,LoyPrgrmCode,LoyPrgrmNumber,AgentInfo
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,OVB,...,7360415302044672,9375 053270,NaN,NaN,J,JGRPGN0PC,S,SU,38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,SVO,...,7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,9,FB,284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,SVO,...,5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,F,NaN,NaN,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,...,5274206497242737,98 6865148,NaN,NaN,J,JFLXLS0PC,#,FB,884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,...,6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,SU,183142068,OZON.travel
5,ГЛУШКОВ КОНСТАНТИН ИЛЬИЧ,NaN,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,SVO,...,5874178506968181,4788 422492,NaN,NaN,Y,YGRPPN0PC,8,FB,553284496,KupiBilet
6,КАПУСТИН АРТЁМ ЭДУАРДОВИЧ,1982-10-24,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,SVO,...,7467749130398378,0058 142289,NaN,VLML,Y,YFLXNM2PC,F,NaN,NaN,KupiBilet
7,ЕРШОВА ЛЮБОВЬ ЗАХАРОВНА,NaN,2017-03-12,11:45,2017-03-12,12:25,SU6284,NO,UUS,SVO,...,2183161939566868,0776 380126,NaN,NaN,Y,YGRPYX0PC,#,NaN,NaN,Aeroflot
8,ТИТОВА ЗАРИНА ЭМИЛЬЕВНА,NaN,2017-07-29,14:15,2017-07-29,15:45,SU1281,NO,KZN,SVO,...,5954073786122008,53 7554162,NaN,NaN,Y,YSTNXJ1PC,8,FB,933538031,OZON.travel
9,БАРСУКОВ САМИР ГОРДЕЕВИЧ,NaN,2017-07-29,14:15,2017-07-29,15:45,SU1281,NO,KZN,SVO,...,2264717979478322,0078 271703,NaN,NaN,Y,YGRPPP1PC,F,NaN,NaN,Aeroflot


In [18]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('Sirena-export-fixed-tab.csv', sep=',')

# Удаляем пробелы в названиях столбцов
#df.columns = df.columns.str.strip()

filtered_df = (df.groupby('Document')
               .filter(lambda x: x['PaxName'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates()

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersTAB.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

                         PaxName  Birth Date Flight Date FlightTime  \
93152    ГЛЕБОВ ГОРДЕЙ МАКАРЕВИЧ  1982-12-31  2017-01-16      01:45   
150692  БУРОВ РАМИЛЬ АЛЬБЕРТОВИЧ  1995-08-24  2017-01-03      11:15   

       ArrivalDate ArrivalTime Flight Number Share Code DepartureAirport  \
93152   2017-01-16       04:35        SU1057         NO              MCX   
150692  2017-01-03       13:35        SU1273         NO              KRR   

       ArrivalAirport  ...      TicketNumber     Document Seat  Meal TrvCls  \
93152             SVO  ...  2082229451418171  8248 013778  NaN  KSML      Y   
150692            SVO  ...  7230123462656045  8248 013778  NaN   NaN      A   

       Baggage PaxAdditionalInfo LoyPrgrmCode LoyPrgrmNumber  AgentInfo  
93152   YGRPFD                 F          NaN            NaN    trip.ru  
150692  ASTNLM               NaN          NaN            NaN        NaN  

[2 rows x 21 columns]


4. XLSX. В отдельном ноутбуке было сведение и перевод в CSV формат всех таблиц. В этом файле нет колонки с документами, поэтому происходит сравнение на билет. (ну пусть будет)

In [20]:
import pandas as pd

# Загрузка данных из CSV файла с указанием разделителя
df = pd.read_csv('xlsx_data.csv', sep=',')

# Удаляем пробелы в названиях столбцов
#df.columns = df.columns.str.strip()

filtered_df = (df.groupby('BookingCode')
               .filter(lambda x: x['Full Name'].nunique() > 1))

# Удаляем дубликаты для удобства
final_df = filtered_df.drop_duplicates()

if final_df.empty:
    print("Совпадений не найдено")
else:
    
    # Выгрузка совпадений в отдельный файл
    final_df.to_csv('DuplicatesPassengersXLSX.csv', index=False) 
    
    # Выводим итоговый DataFrame
    print(final_df)

                    Full Name  SEQUENCE Flight Number DepartureCity  \
737             DANIEL AKIMOV        42        AF1306         PARIS   
2251    KONSTANTIN N GORBUNOV         2        SU1421   CHELYABINSK   
2433              ROGOVA ALLA        59        AF1194         PARIS   
2630          DARIA BESSONOVA         6         SV599         DUBAI   
2822          KULAKOV GEORGIY        41        UX1094     AMSTERDAM   
...                       ...       ...           ...           ...   
612729    MAR'YANA F SUVOROVA        87        DL1160    ATLANTA GA   
612770         DARIA SOLOVEVA         8         SU265         TOKYO   
614068         NEKRASOV IGNAT         2        KE1006         SEOUL   
614592    ADELINA A SNEGIREVA        35         RO302     FRANKFURT   
615583        GLUKHOV VADIM R        30         CI120        TAIPEI   

       DepartureAirport  ArrivalCity ArrivalAirport FlightTime Flight Date  \
737                 CDG  DUESSELDORF            DUS      07:15  2017-